<a href="https://colab.research.google.com/github/MitjaGo/YT/blob/main/ID3TAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install mutagen ipywidgets

from mutagen.easyid3 import EasyID3
from mutagen.id3 import ID3, APIC, ID3NoHeaderError, error
from google.colab import files
from IPython.display import display, Image
import ipywidgets as widgets
import zipfile
from datetime import datetime
import re

# -------------------------------------------
# STEP 1: Upload default thumbnail
# -------------------------------------------
print("📸 Upload a default JPG/PNG thumbnail for all MP3s:")
image_upload = files.upload()
img_name, default_img_data = next(iter(image_upload.items()))
display(Image(data=default_img_data, width=150))

# -------------------------------------------
# STEP 2: Upload MP3 files
# -------------------------------------------
print("\n⬆️ Upload up to 50 MP3 files:")
upload = files.upload()
mp3_files = list(upload.keys())[:50]
print(f"✅ Uploaded {len(mp3_files)} MP3 files.")

# -------------------------------------------
# Helper function: Embed thumbnail into MP3
# -------------------------------------------
def embed_thumbnail(mp3_file, image_data):
    try:
        audio_id3 = ID3(mp3_file)
    except ID3NoHeaderError:
        audio_id3 = ID3()
    audio_id3.delall("APIC")  # remove existing album art
    audio_id3.add(APIC(
        encoding=3,
        mime="image/jpeg",
        type=3,
        desc="Cover",
        data=image_data
    ))
    audio_id3.save(mp3_file)

# -------------------------------------------
# STEP 3: Apply default thumbnail to all MP3s
# -------------------------------------------
for f in mp3_files:
    embed_thumbnail(f, default_img_data)
print("🖼️ Default thumbnail embedded into all MP3 files.")

# -------------------------------------------
# STEP 4: Create editable widgets per file
# -------------------------------------------
file_widgets = []

for f in mp3_files:
    try:
        audio = EasyID3(f)
        title = audio.get("title", [""])[0]
        artist = audio.get("artist", [""])[0]
        album = audio.get("album", [""])[0]
    except (error, ID3NoHeaderError):
        title = artist = album = ""

    title_widget = widgets.Text(value=title, description='Title:', layout=widgets.Layout(width='250px'))
    artist_widget = widgets.Text(value=artist, description='Artist:', layout=widgets.Layout(width='250px'))
    album_widget = widgets.Text(value=album, description='Album:', layout=widgets.Layout(width='250px'))
    img_widget = widgets.Image(value=default_img_data, format='jpeg', width=80, height=80)

    # Store new thumbnail here
    new_img_data = {'data': None}

    # Button to upload per-file thumbnail
    def make_upload_callback(img_w, store):
        def on_upload(change):
            uploaded = files.upload()
            if uploaded:
                _, data = next(iter(uploaded.items()))
                img_w.value = data
                store['data'] = data
        return on_upload

    upload_btn = widgets.Button(description="📤 Change Thumbnail", layout=widgets.Layout(width='150px'))
    upload_btn.on_click(make_upload_callback(img_widget, new_img_data))

    hbox = widgets.HBox([img_widget, widgets.Label(f, layout=widgets.Layout(width='200px')),
                         title_widget, artist_widget, album_widget, upload_btn])

    file_widgets.append((f, title_widget, artist_widget, album_widget, img_widget, new_img_data, hbox))

display(widgets.Label("📝 Edit MP3 Tags and optionally upload new thumbnails:"))
for w in file_widgets:
    display(w[-1])

# -------------------------------------------
# STEP 5: Save & Download buttons
# -------------------------------------------
save_btn = widgets.Button(description="💾 Save All Tags", button_style='success')
zip_btn  = widgets.Button(description="⬇️ Download All MP3s as ZIP", button_style='info')
output   = widgets.Output()
display(save_btn, zip_btn, output)

# -------------------------------------------
# Helper function: Clean filenames (remove any number in parentheses)
# -------------------------------------------
def clean_filename(filename):
    # Remove trailing (number) before extension
    return re.sub(r"\s*\(\d+\)(?=\.\w+$)", "", filename)

# -------------------------------------------
# Save button function
# -------------------------------------------
def on_save_clicked(b):
    for f, title_w, artist_w, album_w, img_w, new_img, _ in file_widgets:
        # Update ID3 tags
        try:
            audio = EasyID3(f)
        except ID3NoHeaderError:
            audio = EasyID3()
        audio['title']  = title_w.value.strip() or ""
        audio['artist'] = artist_w.value.strip() or ""
        audio['album']  = album_w.value.strip() or ""
        audio.save()

        # Embed thumbnail (default or new)
        embed_thumbnail(f, new_img['data'] if new_img['data'] else default_img_data)

    with output:
        output.clear_output()
        print("✅ All MP3 tags and thumbnails updated successfully!")

# -------------------------------------------
# ZIP download button function
# -------------------------------------------
def on_zip_clicked(b):
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    zip_name = f"edited_mp3s_{now}.zip"
    with zipfile.ZipFile(zip_name, "w") as z:
        for f, *_ in file_widgets:
            new_name = clean_filename(f)
            z.write(f, arcname=new_name)
    files.download(zip_name)

save_btn.on_click(on_save_clicked)
zip_btn.on_click(on_zip_clicked)


📸 Upload a default JPG/PNG thumbnail for all MP3s:
